In [ ]:
# Import libraries
import csv
import re
import pandas as pd

In [ ]:
# Define files names
original_file = 'movies.csv'
cleaned_file_year = 'movies_year.csv'  # New CSV file with year
cleaned_file_genres = 'movies_genres.csv'  # New CSV file with split genres

In [ ]:
# Open the original CSV file for reading
with open(original_file, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)

    # Read first row as header
    header = next(reader)

    # Create new headers for cleaned_file_year csv file without genres and adding year
    genres_index = header.index('genres')
    cleaned_header = header[:genres_index] + ['year']

    # Create headers for cleaned_file_genres csv file
    genres_header = ['movieId', 'genre']

    # Open new files for writing
    with open(cleaned_file_year, 'w', newline='', encoding='utf-8') as new_file, \
         open(cleaned_file_genres, 'w', newline='', encoding='utf-8') as genres_file:

        year_writer = csv.writer(new_file)
        genres_writer = csv.writer(genres_file)

        # Write the new headers for each file
        year_writer.writerow(cleaned_header)
        genres_writer.writerow(genres_header)

        # Loop over every row in the original CSV file
        for row in reader:
            title = row[1]

            # Year column in cleaned_file_year csv file
            year_match = re.search(r'\((\d{4})\)', title)

            if year_match:
                year = year_match.group(1)
                title_without_year = re.sub(r' \(\d{4}\)', '', title)
            else:
                title_without_year = title
                year = ''

            year_writer.writerow([row[0], title_without_year, year])

            # Genres column in cleaned_file_genres csv file
            genres = row[genres_index]
            genre_list = genres.split('|')
            for genre in genre_list:
                genres_writer.writerow([row[0], genre])


In [ ]:
# Load data (replace with the actual path to your CSV or data source)
ratings_df = pd.read_csv('/content/ratings.csv')
movies_year_df = pd.read_csv('/content/movies_year.csv')

# Merge ratings and movie titles
merged_df = pd.merge(ratings_df, movies_year_df, on='movieId')

# Group by movieId and title, calculate the average rating and sort to get the top 10
top_10_movies = merged_df.groupby(['movieId', 'title'])['rating'].mean().reset_index()
top_10_movies = top_10_movies.sort_values(by='rating', ascending=False).head(10)

print("Top 10 Rated Movies:")
print(top_10_movies)


Top 10 Rated Movies:
      movieId                                        title  rating
4239     8955                                     Undertow     5.0
5737    92494                         Dylan Moran: Monster     5.0
5749    93320                            Trailer Park Boys     5.0
5750    93324                                   Undefeated     5.0
1180     1824                                    Homegrown     5.0
256       326                             To Live (Huozhe)     5.0
4025     7826             Secret Life of Walter Mitty, The     5.0
1198     1859             Taste of Cherry (Ta'm e guilass)     5.0
4013     7773                      Bang, Bang, You're Dead     5.0
3997     7700  Wages of Fear, The (Salaire de la peur, Le)     5.0


In [ ]:
# Load genre data (replace with actual path)
movies_genres_df = pd.read_csv('/content/movies_genres.csv')

# Merge genres and ratings
merged_genre_df = pd.merge(movies_genres_df, ratings_df, on='movieId')

# Group by genre and calculate the average rating per genre
top_10_genres = merged_genre_df.groupby('genre')['rating'].mean().reset_index()
top_10_genres = top_10_genres.sort_values(by='rating', ascending=False).head(10)

print("Top 10 Genres by Rating:")
print(top_10_genres)


Top 10 Genres by Rating:
                 genre    rating
0   (no genres listed)  4.166667
10           Film-Noir  4.011983
18                 War  3.848165
7          Documentary  3.717694
8                Drama  3.715236
6                Crime  3.692476
14             Mystery  3.688889
19             Western  3.641332
3            Animation  3.609075
15             Romance  3.603037
